In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import random
import matplotlib.image as mpimg
import splitfolders
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from keras.models import Sequential
from keras import models
from keras.layers import Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
import shutil
import torch
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from IPython.display import SVG
from scipy import ndimage
import pathlib as Path
from roboflow import Roboflow

In [2]:
!python '/home/lunet/conm/Desktop/Stenosis-Project/yolov5/{'train.py'}' --img 700 --batch 32 --epochs 100 --data {'/home/lunet/conm/Desktop/Stenosis-Project/yolov5/Stenosis-7'}/data.yaml --weights yolov5s.pt --cache ram

train: weights=yolov5s.pt, cfg=, data=/home/lunet/conm/Desktop/Stenosis-Project/yolov5/Stenosis-7/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=32, imgsz=700, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 1), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/ultralytics/yolov5
   08c8c3e..e5991c9  master     -> origin/master
   d250559..606ed8f  u

In [5]:
!python '/home/lunet/conm/Desktop/Stenosis-Project/yolov5/{'detect.py'}' --weights {'/home/lunet/conm/Desktop/Stenosis-Project/yolov5/runs/train/exp10/weights'}/best.pt --img 700 --conf 0.1 --source {'/home/lunet/conm/Desktop/Stenosis-Project/yolov5/Stenosis-7'}/test/images

detect: weights=['/home/lunet/conm/Desktop/Stenosis-Project/yolov5/runs/train/exp10/weights/best.pt'], source=/home/lunet/conm/Desktop/Stenosis-Project/yolov5/Stenosis-7/test/images, data=yolov5/data/coco128.yaml, imgsz=[700, 700], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-348-g0669f1b Python-3.10.4 torch-1.12.0+cu102 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11016MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/833 /home/lunet/conm/Desktop/Stenosis-Project/yolov5/Stenosis-7/test/images/14_002_5_0018_bmp.rf.375c38509c58df2054928b3c375b8f34.jpg: 704x704 1 Stenosis, Done. (0.005s)
image 2/833 /home/lunet/conm/Desktop/Stenosis-Pr

In [13]:
if torch.cuda.is_available():
    print('GPU is available')
else:
    device = torch.device("cpu")
    print(device)

GPU is available


In [9]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.12.0+cu102 (NVIDIA GeForce RTX 2080 Ti)
